In [1]:
import pandas as pd
from helper.constantes import *
from ast import literal_eval

In [2]:
networks = pd.read_csv(cleaned_folder+"selected_chars.csv",converters={"associated_groups": literal_eval,"connection_label": literal_eval})

In [67]:
networks = networks.rename(columns={'Unnamed: 0':"id"})

In [68]:
networks_assoc = networks.loc[:,['name','associated_groups']]
networks_inter = networks.loc[:,['name','connection_label']]

In [69]:
networks_assoc.head()

,name,associated_groups
0,Aragog,[]
1,Ludovic Bagman,[British Ministry of Magic]
2,Bane,[Hogwarts School of Witchcraft and Wizardry]
3,Cuthbert Binns,[Hogwarts School of Witchcraft and Wizardry]
4,Regulus Black,"[Slytherin, Hogwarts School of Witchcraft and ..."


In [70]:
all_assoc = {}


def add_assoc_to_dico(index, list_groups, dico):
    for l in list_groups:
        if l in dico: 
            cur_set = dico[l]
        else:
            cur_set = set()
        cur_set.add(index)
        dico[l] = cur_set


In [134]:
networks_assoc.apply(lambda x: add_assoc_to_dico(x.name,x['associated_groups'],all_assoc),axis=1)


0      None
1      None
2      None
3      None
4      None
       ... 
171    None
172    None
173    None
174    None
175    None
Length: 176, dtype: object

In [136]:
del all_assoc["Hogwarts School of Witchcraft and Wizardry"]

In [137]:
all_perm = [(x,y,k) for k,v in all_assoc.items() for x in list(v) for y in list(v)  if x != y]
edges_assoc = pd.DataFrame(all_perm, columns=['source', 'target', 'association']).groupby(['source','target'])['association'].apply(list).reset_index()

In [138]:
def add_inter_to_dico(index, list_inters, dico, map_name_to_index):
    all_inter = set()
    for l in list_inters:
        all_inter.add(map_name_to_index[l])
    dico[index] = all_inter

In [139]:
map_name_to_index = {}

for index, row in networks_inter.iterrows():
    map_name_to_index[row['name']]= index

In [140]:
all_inters = {}

networks_inter.apply(lambda x: add_inter_to_dico(x.name,x['connection_label'],all_inters,map_name_to_index),axis=1)
all_perm_inter = [(k,x) for k,v in all_inters.items() for x in list(v)]
edges_inter = pd.DataFrame(all_perm_inter, columns=['source', 'target'])


In [141]:
len(edges_inter[edges_inter.source == 140].values)

49

In [142]:
len(networks_inter[networks_inter.name == "Bellatrix Lestrange"]['connection_label'].values[0])

49

In [143]:
networks_inter[networks_inter.name == "Bellatrix Lestrange"]

,name,connection_label
140,Bellatrix Lestrange,"[Aberforth Dumbledore, Alastor Moody, Albus Du..."


In [144]:
edges_inter.to_csv(cleaned_folder+"edges_inter.csv")

In [145]:
edges_assoc.to_csv(cleaned_folder+"edges_assoc.csv")

### Generate graph in networkx

In [146]:
import networkx as nx
from networkx.readwrite import json_graph
import json

In [150]:
def add_nodes_attributes_and_save(filepath, edge_list, node_info):
    edge_list = edge_list.reset_index().rename(columns={'index':'id'})
    g=nx.from_pandas_edgelist(edge_list, source='source', target='target',edge_attr='id')
    node_info.fillna("",inplace=True)
    
    node_attr = node_info.to_dict(orient='index') 
    nx.set_node_attributes(g, node_attr)
    nx.set_node_attributes(g,{k:{"x":v[0], "y":v[1], "color":"#EEEEEE","size":2} for k,v in nx.spring_layout(g).items()})
    
    json_net = json_graph.node_link_data(g)
    edges = json_net['links']
    json_net["edges"]=edges.copy()
    del json_net['links']
    
    with open(filepath, 'w') as f:
        json.dump(json_net, f)

In [151]:
add_nodes_attributes_and_save(cleaned_folder + "interaction_network.json", edges_inter, networks)

/var/folders/vf/btrykr5n65v_k_lwb61317k80000gn/T/ipykernel_33540/4178445733.py:6: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  node_attr = node_info.to_dict(orient='index')


In [152]:
add_nodes_attributes_and_save(cleaned_folder + "association_network.json", edges_assoc, networks)

/var/folders/vf/btrykr5n65v_k_lwb61317k80000gn/T/ipykernel_33540/4178445733.py:6: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  node_attr = node_info.to_dict(orient='index')
